In [1]:
import numpy as np
import random as rd
import time
import heapq

In [74]:
file1 = open('Sample Tours/reuc_100')
file2 = open('Sample Tours/rnoneuc_100')
file3 = open('Sample Tours/reuc_500')
cont = file3.read().split('\n')
file1.close();file2.close();file3.close()

In [75]:
eu = (cont[0] == 'euclidean')
n = int(cont[1])
points = []
for i in range(n):
    points.append(list(map(float,cont[i+2].split())))
dis = []
for i in range(n,2*n):
    dis.append(list(map(float,cont[i+2].split())))

In [76]:
def travel(goto):
    global dis,n
    ans = 0
    for i in range(n):
        ans += dis[i][goto[i]]
    return ans
def find(i,parent):
    if i == parent[i]:
        return i
    return find(parent[i],parent)
def union(i,j,parent,rank):
    i = find(i,parent)
    j = find(j,parent)
    if i == j:
        return False
    if rank[i] > rank[j]:
        parent[j] = i
    elif rank[j] > rank[i]:
        parent[i] = j
    else:
        rank[i]+=1
        parent[j] = i
    return True
def check_goto(goto):
    global n
    visited = [0]*n
    count = 0
    x = goto[0]
    visited[0] = 1
    while visited[x] == 0:
        visited[x] = 1
        x = goto[x]
        count+=1
    return count == n-1
def get_ind(v,fit,p):
    l = 0
    r = p
    while (l < r):
        m = (l+r)//2
        if v > fit[m]:
            l = m+1
        else:
            r = m
    return l

In [78]:
# edge exchange + savings
start = time.time()
ver = [i for i in range(n)]
results = []
for s in ver:
    cost = []
    for i in range(n):
        for j in range(i+1,n):
            if i != s and j != s:
                cost.append((dis[i][s]+dis[j][s]-dis[i][j],i,j))
    cost.sort(reverse=True)
    chain = [[] for i in range(n)]
    parent = [i for i in range(n)]
    rank = [0]*n
    e = 0;i = 0
    while e < n-2:
        if len(chain[cost[i][1]]) < 2 and len(chain[cost[i][2]]) < 2:
            if union(cost[i][1],cost[i][2],parent,rank):
                chain[cost[i][1]].append(cost[i][2])
                chain[cost[i][2]].append(cost[i][1])
                e+=1
        i+=1
    for i in range(n):
        if i!=s and len(chain[i]) < 2:
            chain[i].append(s)
            chain[s].append(i)
    goto = [-1]*n
    goback = [-1]*n
    par = s
    ch = chain[s][0]
    while ch != s:
        goto[par] = ch
        goback[ch] = par
        x = chain[ch][0]
        if x == par:
            x = chain[ch][1]
        par = ch
        ch = x
    goto[par] = ch
    goback[ch] = par
    while True:
        diff = 0;v1 = -1;v2 = -1
        for i in range(n):
            for j in range(i+1,n):
                if goto[i] != j and goto[j] != i:
                    d1 = dis[i][goto[i]]+dis[j][goto[j]]
                    d2 = dis[i][j]+dis[goto[i]][goto[j]]
                    if diff < d1-d2:
                        v1 = i;v2 = j;diff = d1-d2
        if diff == 0:
            break
        else:
            gt1 = goto[v1];gt2 = goto[v2]
            x = v1;y = v2
            while x != gt1:
                goto[x] = y;var = goback[y];goback[y] = x
                x = y;y = var
            goto[gt1] = gt2;goback[gt2] = gt1
    results.append((travel(goto),goto))
    # print(results[-1][0],check_goto(goto))
    # print(time.time()-start)
    # start = time.time()
print(min(results))

(3461.1148166691173, [326, 182, 480, 466, 163, 349, 299, 316, 89, 498, 418, 309, 188, 15, 452, 456, 55, 28, 212, 264, 463, 8, 459, 174, 129, 351, 364, 291, 314, 425, 268, 245, 373, 194, 114, 34, 98, 252, 225, 140, 186, 484, 197, 273, 37, 388, 72, 359, 279, 389, 491, 417, 486, 13, 253, 150, 431, 255, 453, 493, 344, 398, 355, 450, 411, 21, 424, 257, 178, 455, 311, 380, 433, 356, 36, 243, 435, 428, 320, 347, 379, 79, 242, 315, 124, 120, 369, 283, 9, 187, 446, 146, 166, 412, 154, 105, 176, 336, 426, 148, 408, 384, 407, 43, 234, 63, 168, 370, 397, 313, 301, 328, 119, 335, 137, 67, 239, 81, 473, 230, 488, 329, 70, 107, 85, 5, 12, 289, 233, 93, 427, 223, 405, 338, 443, 307, 128, 258, 232, 131, 442, 46, 334, 83, 423, 92, 414, 476, 310, 100, 286, 469, 136, 82, 276, 284, 421, 482, 183, 394, 294, 333, 432, 224, 306, 20, 222, 127, 18, 190, 61, 29, 135, 143, 106, 165, 145, 74, 481, 193, 312, 396, 261, 68, 235, 156, 141, 95, 38, 474, 27, 317, 58, 457, 236, 434, 250, 4, 354, 350, 115, 437, 440, 117, 

In [77]:
# greedy edge selection + HC
cost = []
for i in range(n):
    for j in range(i+1,n):
        cost.append((dis[i][j],i,j))
cost.sort()
parent = [i for i in range(n)]
rank = [0]*n
chain = [[] for i in range(n)]
e = 0;i = 0
while e < n-1:
    if len(chain[cost[i][1]]) < 2 and len(chain[cost[i][2]]) < 2:
        if union(cost[i][1],cost[i][2],parent,rank):
            chain[cost[i][1]].append(cost[i][2])
            chain[cost[i][2]].append(cost[i][1])
            e+=1
    i+=1
x = -1
for i in range(n):
    if len(chain[i]) < 2:
        if x == -1:
            x = i
        else:
            chain[x].append(i)
            chain[i].append(x)
goto = [-1]*n
goback = [-1]*n
par = s
ch = chain[s][0]
while ch != s:
    goto[par] = ch
    goback[ch] = par
    x = chain[ch][0]
    if x == par:
        x = chain[ch][1]
    par = ch
    ch = x
goto[par] = ch
goback[ch] = par
gt = goto.copy()
gb = goback.copy()
while True:
    diff = 0;v1 = -1;v2 = -1
    for i in range(n):
        for j in range(i+1,n):
            if goto[i] != j and goto[j] != i:
                d1 = dis[i][goto[i]]+dis[j][goto[j]]
                d2 = dis[i][j]+dis[goto[i]][goto[j]]
                if diff < d1-d2:
                    v1 = i;v2 = j;diff = d1-d2
    if diff == 0:
        break
    else:
        gt1 = goto[v1];gt2 = goto[v2]
        x = v1;y = v2
        while x != gt1:
            goto[x] = y;var = goback[y];goback[y] = x
            x = y;y = var
        goto[gt1] = gt2;goback[gt2] = gt1
print(travel(goto),check_goto(goto))

3465.2155267261664 True


In [7]:
# 2 vertice exchange Neighbour
results = []
for _ in range(1):
    start = time.time()
    rand_arr = [i for i in range(n)]
    rd.shuffle(rand_arr)
    goto = [-1]*n
    goback = [-1]*n
    for i in range(n-1):
        goto[rand_arr[i]] = rand_arr[i+1]
        goback[rand_arr[i+1]] = rand_arr[i]
    goto[rand_arr[n-1]] = rand_arr[0]
    goback[rand_arr[0]] = rand_arr[n-1]
    while True:
        diff = 0;v1 = -1;v2 = -1
        for i in range(n):
            for j in range(i+1,n):
                if goto[i] != j and goto[j] != i:
                    d1 = dis[i][goto[i]]+dis[goback[i]][i]+dis[j][goto[j]]+dis[goback[j]][j]
                    d2 = dis[goback[i]][j]+dis[j][goto[i]]+dis[goback[j]][i]+dis[i][goto[j]]
                    if diff < d1-d2:
                        v1 = i;v2 = j;diff = d1-d2
        if diff == 0:
            break
        else:
            gt1,gb1,gt2,gb2 = goto[v1],goback[v1],goto[v2],goback[v2]
            goto[gb1] = v2;goto[v2] = gt1;goto[gb2] = v1;goto[v1] = gt2
            goback[v2] = gb1;goback[gt1] = v2;goback[v1] = gb2;goback[gt2] = v1
    results.append((travel(goto),goto))
    print(time.time()-start)
    print(results[-1][0],check_goto(results[-1][1]))
# print(min(results))

0.6273598670959473
3163.2076379342097 True


In [1]:
"Normal"
164.71859455108643
9100.718585892204, True
"Break"
165.60947465896606
9316.2573741958, True

(9316.2573741958, True)

In [8]:
# 2 edge exchange neighbour - iterated HC with n different start points
results = []
for _ in range(10):
    start = time.time()
    rand_arr = [i for i in range(n)]
    rd.shuffle(rand_arr)
    goto = [-1]*n
    goback = [-1]*n
    for i in range(n-1):
        goto[rand_arr[i]] = rand_arr[i+1]
        goback[rand_arr[i+1]] = rand_arr[i]
    goto[rand_arr[n-1]] = rand_arr[0]
    goback[rand_arr[0]] = rand_arr[n-1]
    while True:
        diff = 0;v1 = -1;v2 = -1
        for i in range(n):
            for j in range(i+1,n):
                if goto[i] != j and goto[j] != i:
                    d1 = dis[i][goto[i]]+dis[j][goto[j]]
                    d2 = dis[i][j]+dis[goto[i]][goto[j]]
                    if diff < d1-d2:
                        v1 = i;v2 = j;diff = d1-d2
        if diff == 0:
            break
        else:
            gt1 = goto[v1];gt2 = goto[v2]
            x = v1;y = v2
            while x != gt1:
                goto[x] = y;var = goback[y];goback[y] = x
                x = y;y = var
            goto[gt1] = gt2;goback[gt2] = gt1
    results.append((travel(goto),goto))
    # print(time.time()-start)
    # print(results[-1][0],check_goto(results[-1][1]))
print(min(results))

(1586.8595340635104, [21, 28, 64, 96, 6, 97, 8, 56, 84, 20, 4, 37, 65, 5, 42, 72, 81, 62, 93, 67, 11, 80, 19, 98, 83, 45, 95, 50, 12, 48, 17, 54, 85, 46, 61, 53, 86, 47, 30, 52, 66, 90, 77, 82, 58, 57, 78, 49, 25, 59, 9, 2, 35, 68, 1, 23, 87, 88, 63, 32, 99, 79, 27, 31, 69, 43, 14, 41, 71, 15, 26, 91, 18, 60, 92, 38, 34, 55, 13, 10, 24, 51, 73, 94, 39, 74, 33, 3, 44, 7, 75, 36, 29, 70, 40, 89, 0, 22, 76, 16])


In [15]:
# hill climbing with beam
b = 100
rand_arr = [i for i in range(n)]
goto_arr_main = []
for i in range(b):
    rd.shuffle(rand_arr)
    goto = [-1]*n
    goback = [-1]*n
    for i in range(n-1):
        goto[rand_arr[i]] = rand_arr[i+1]
        goback[rand_arr[i+1]] = rand_arr[i]
    goto[rand_arr[n-1]] = rand_arr[0]
    goback[rand_arr[0]] = rand_arr[n-1]
    goto_arr_main.append((travel(goto),goto,goback))
print(min(goto_arr_main)[:2])

(9237.78744770338, [63, 75, 25, 43, 31, 18, 56, 24, 33, 30, 60, 86, 45, 23, 6, 52, 47, 34, 73, 87, 48, 49, 94, 10, 76, 26, 64, 66, 97, 35, 78, 7, 29, 99, 82, 19, 27, 38, 51, 61, 28, 36, 90, 13, 77, 1, 17, 22, 88, 14, 15, 69, 54, 83, 5, 62, 89, 68, 92, 96, 93, 55, 9, 67, 20, 50, 44, 84, 81, 16, 8, 41, 70, 42, 46, 32, 39, 40, 85, 98, 2, 79, 57, 72, 58, 11, 53, 0, 21, 95, 4, 65, 71, 80, 91, 37, 74, 59, 3, 12])


In [60]:
Q = 10000
b = 10
keep = 6
av = 0
for _ in range(100):
    start = time.time()
    goto_arr = [(travel(gt),gt.copy(),gb.copy()) for i in range(b)]
    while True:
        hp = []
        for k in range(len(goto_arr)):
            tr,goto,goback = goto_arr[k]
            for i in range(n):
                for j in range(i+1,n):
                    if goto[i] != j and goto[j] != i:
                        d1 = dis[i][goto[i]]+dis[j][goto[j]]
                        d2 = dis[i][j]+dis[goto[i]][goto[j]]
                        if d1-d2 > 0:
                            hp.append((tr+d2-d1,i,j,k))
        if hp:
            new_goto = []
            hp.sort()
            p = len(hp)-keep
            if p > 0:
                select = [i for i in range(keep)]
                fit = [0]*p
                tot = 0
                for i in range(p):
                    fit[i] = Q/hp[i+keep][0]
                    tot += fit[i]
                fit[0] = fit[0]/tot
                for i in range(1,p):
                    fit[i] = fit[i]/tot
                    fit[i] += (fit[i-1])
                for i in range(keep,len(goto_arr)):
                    select.append(get_ind(rd.random(),fit,p)+keep)
            else:
                select = [i for i in range(len(hp))]
            for k in select:
                goto = goto_arr[hp[k][3]][1].copy()
                goback = goto_arr[hp[k][3]][2].copy()
                v1 = hp[k][1];v2 = hp[k][2]
                gt1 = goto[v1];gt2 = goto[v2]
                x = v1;y = v2
                while x != gt1:
                    goto[x] = y;var = goback[y];goback[y] = x
                    x = y;y = var
                goto[gt1] = gt2;goback[gt2] = gt1
                new_goto.append((hp[k][0],goto,goback))
            goto_arr = new_goto
        else:
            break
    print(time.time()-start,min(goto_arr)[:2])
    av += min(goto_arr)[0]
print(av/100)

0.6587855815887451 (1509.5102024155208, [21, 54, 64, 81, 6, 27, 8, 56, 84, 47, 4, 20, 23, 19, 42, 72, 99, 30, 93, 22, 37, 80, 97, 98, 83, 48, 95, 50, 12, 85, 38, 63, 59, 36, 61, 1, 91, 32, 58, 52, 66, 68, 73, 65, 88, 25, 78, 11, 29, 17, 9, 2, 35, 86, 31, 14, 87, 45, 44, 49, 0, 79, 5, 90, 69, 28, 77, 41, 71, 15, 26, 53, 18, 82, 92, 67, 34, 55, 13, 10, 24, 16, 43, 94, 39, 74, 33, 96, 57, 7, 75, 46, 51, 70, 40, 89, 3, 62, 76, 60])
0.5744290351867676 (1501.8991408834709, [60, 35, 51, 96, 10, 62, 4, 89, 6, 50, 79, 47, 28, 33, 55, 69, 81, 49, 72, 78, 11, 0, 19, 12, 80, 45, 70, 5, 65, 48, 17, 54, 37, 46, 76, 52, 86, 20, 30, 84, 94, 67, 14, 82, 58, 57, 91, 9, 25, 59, 27, 92, 39, 71, 1, 77, 7, 88, 38, 32, 99, 34, 97, 31, 2, 43, 40, 75, 41, 64, 93, 68, 15, 42, 85, 90, 98, 66, 13, 61, 21, 3, 73, 24, 8, 29, 53, 56, 44, 95, 63, 36, 74, 18, 83, 26, 87, 22, 23, 16])
0.49471545219421387 (1509.510202415521, [21, 54, 64, 81, 6, 27, 8, 56, 84, 47, 4, 20, 23, 19, 42, 72, 99, 30, 93, 22, 37, 80, 97, 98, 83

0.578453779220581 (1501.8991408834709, [60, 35, 51, 96, 10, 62, 4, 89, 6, 50, 79, 47, 28, 33, 55, 69, 81, 49, 72, 78, 11, 0, 19, 12, 80, 45, 70, 5, 65, 48, 17, 54, 37, 46, 76, 52, 86, 20, 30, 84, 94, 67, 14, 82, 58, 57, 91, 9, 25, 59, 27, 92, 39, 71, 1, 77, 7, 88, 38, 32, 99, 34, 97, 31, 2, 43, 40, 75, 41, 64, 93, 68, 15, 42, 85, 90, 98, 66, 13, 61, 21, 3, 73, 24, 8, 29, 53, 56, 44, 95, 63, 36, 74, 18, 83, 26, 87, 22, 23, 16])
0.5694777965545654 (1501.899140883471, [21, 54, 64, 81, 6, 27, 8, 56, 84, 47, 4, 20, 23, 78, 42, 72, 99, 30, 93, 22, 37, 80, 97, 98, 83, 48, 95, 50, 12, 85, 38, 63, 59, 13, 61, 1, 91, 32, 58, 52, 66, 68, 73, 65, 88, 25, 33, 11, 29, 17, 9, 2, 35, 86, 31, 14, 87, 45, 44, 49, 0, 79, 5, 90, 69, 28, 77, 41, 71, 15, 26, 53, 18, 82, 92, 67, 34, 55, 19, 10, 24, 16, 43, 94, 39, 74, 36, 96, 57, 7, 75, 46, 51, 70, 40, 89, 3, 62, 76, 60])
0.49989771842956543 (1509.510202415521, [21, 54, 64, 81, 6, 27, 8, 56, 84, 47, 4, 20, 23, 19, 42, 72, 99, 30, 93, 22, 37, 80, 97, 98, 83, 

0.4946770668029785 (1509.510202415521, [21, 54, 64, 81, 6, 27, 8, 56, 84, 47, 4, 20, 23, 19, 42, 72, 99, 30, 93, 22, 37, 80, 97, 98, 83, 48, 95, 50, 12, 85, 38, 63, 59, 36, 61, 1, 91, 32, 58, 52, 66, 68, 73, 65, 88, 25, 78, 11, 29, 17, 9, 2, 35, 86, 31, 14, 87, 45, 44, 49, 0, 79, 5, 90, 69, 28, 77, 41, 71, 15, 26, 53, 18, 82, 92, 67, 34, 55, 13, 10, 24, 16, 43, 94, 39, 74, 33, 96, 57, 7, 75, 46, 51, 70, 40, 89, 3, 62, 76, 60])
0.6612691879272461 (1509.5102024155208, [21, 54, 64, 81, 6, 27, 8, 56, 84, 47, 4, 20, 23, 19, 42, 72, 99, 30, 93, 22, 37, 80, 97, 98, 83, 48, 95, 50, 12, 85, 38, 63, 59, 36, 61, 1, 91, 32, 58, 52, 66, 68, 73, 65, 88, 25, 78, 11, 29, 17, 9, 2, 35, 86, 31, 14, 87, 45, 44, 49, 0, 79, 5, 90, 69, 28, 77, 41, 71, 15, 26, 53, 18, 82, 92, 67, 34, 55, 13, 10, 24, 16, 43, 94, 39, 74, 33, 96, 57, 7, 75, 46, 51, 70, 40, 89, 3, 62, 76, 60])
0.4926464557647705 (1509.510202415521, [21, 54, 64, 81, 6, 27, 8, 56, 84, 47, 4, 20, 23, 19, 42, 72, 99, 30, 93, 22, 37, 80, 97, 98, 83, 

0.8955702781677246 (1517.6499933762207, [21, 54, 64, 81, 6, 62, 8, 56, 84, 47, 4, 20, 23, 19, 42, 72, 99, 27, 93, 22, 37, 80, 97, 98, 83, 48, 95, 50, 12, 85, 38, 63, 59, 36, 61, 1, 91, 32, 58, 52, 66, 68, 73, 65, 88, 25, 78, 11, 29, 30, 9, 2, 35, 86, 31, 14, 87, 45, 44, 49, 0, 79, 17, 90, 69, 28, 77, 41, 71, 15, 26, 53, 18, 82, 92, 67, 34, 55, 13, 10, 24, 16, 43, 94, 39, 74, 33, 96, 57, 7, 75, 46, 51, 70, 40, 89, 3, 5, 76, 60])
0.6476955413818359 (1520.9164027606505, [21, 54, 64, 81, 6, 27, 8, 87, 84, 47, 4, 20, 23, 78, 42, 72, 99, 30, 93, 22, 37, 80, 97, 98, 83, 85, 95, 50, 12, 48, 38, 63, 59, 13, 61, 1, 91, 32, 58, 52, 66, 68, 73, 65, 88, 25, 33, 11, 51, 17, 9, 2, 35, 86, 31, 14, 89, 45, 44, 49, 0, 79, 5, 90, 69, 28, 77, 41, 71, 15, 26, 53, 18, 82, 92, 67, 34, 55, 19, 10, 24, 16, 43, 94, 39, 74, 36, 96, 57, 7, 75, 46, 29, 70, 40, 56, 3, 62, 76, 60])
0.48669910430908203 (1509.510202415521, [21, 54, 64, 81, 6, 27, 8, 56, 84, 47, 4, 20, 23, 19, 42, 72, 99, 30, 93, 22, 37, 80, 97, 98, 83

0.5574123859405518 (1501.899140883471, [21, 54, 64, 81, 6, 27, 8, 56, 84, 47, 4, 20, 23, 78, 42, 72, 99, 30, 93, 22, 37, 80, 97, 98, 83, 48, 95, 50, 12, 85, 38, 63, 59, 13, 61, 1, 91, 32, 58, 52, 66, 68, 73, 65, 88, 25, 33, 11, 29, 17, 9, 2, 35, 86, 31, 14, 87, 45, 44, 49, 0, 79, 5, 90, 69, 28, 77, 41, 71, 15, 26, 53, 18, 82, 92, 67, 34, 55, 19, 10, 24, 16, 43, 94, 39, 74, 36, 96, 57, 7, 75, 46, 51, 70, 40, 89, 3, 62, 76, 60])
0.4874143600463867 (1509.510202415521, [21, 54, 64, 81, 6, 27, 8, 56, 84, 47, 4, 20, 23, 19, 42, 72, 99, 30, 93, 22, 37, 80, 97, 98, 83, 48, 95, 50, 12, 85, 38, 63, 59, 36, 61, 1, 91, 32, 58, 52, 66, 68, 73, 65, 88, 25, 78, 11, 29, 17, 9, 2, 35, 86, 31, 14, 87, 45, 44, 49, 0, 79, 5, 90, 69, 28, 77, 41, 71, 15, 26, 53, 18, 82, 92, 67, 34, 55, 13, 10, 24, 16, 43, 94, 39, 74, 33, 96, 57, 7, 75, 46, 51, 70, 40, 89, 3, 62, 76, 60])
0.6917283535003662 (1516.540633539871, [60, 35, 51, 96, 10, 62, 4, 89, 6, 50, 79, 47, 28, 33, 55, 69, 81, 49, 72, 78, 11, 0, 19, 12, 80, 4

0.5844416618347168 (1506.2749101042507, [60, 35, 51, 96, 10, 62, 4, 89, 6, 50, 79, 47, 28, 33, 55, 69, 81, 49, 72, 78, 11, 0, 19, 12, 80, 45, 70, 5, 65, 48, 17, 54, 37, 46, 76, 52, 86, 20, 30, 84, 94, 67, 14, 82, 58, 57, 91, 9, 25, 59, 27, 92, 39, 71, 1, 77, 95, 88, 38, 32, 99, 34, 97, 31, 2, 43, 40, 75, 41, 64, 93, 68, 15, 42, 85, 90, 98, 66, 13, 61, 21, 3, 73, 24, 8, 29, 53, 7, 44, 56, 63, 36, 74, 18, 83, 26, 87, 22, 23, 16])
0.5255918502807617 (1513.8859716363008, [21, 54, 64, 81, 6, 27, 8, 87, 84, 47, 4, 20, 23, 19, 42, 72, 99, 30, 93, 22, 37, 80, 97, 98, 83, 48, 95, 50, 12, 85, 38, 63, 59, 36, 61, 1, 91, 32, 58, 52, 66, 68, 73, 65, 88, 25, 78, 11, 29, 17, 9, 2, 35, 86, 31, 14, 89, 45, 44, 49, 0, 79, 5, 90, 69, 28, 77, 41, 71, 15, 26, 53, 18, 82, 92, 67, 34, 55, 13, 10, 24, 16, 43, 94, 39, 74, 33, 96, 57, 7, 75, 46, 51, 70, 40, 56, 3, 62, 76, 60])
0.4916853904724121 (1509.510202415521, [21, 54, 64, 81, 6, 27, 8, 56, 84, 47, 4, 20, 23, 19, 42, 72, 99, 30, 93, 22, 37, 80, 97, 98, 83,

In [ ]:
# set 1
1, 1546
2, 1637
5, 1568
10, 1568
15, 1587
# set 2
100, 1661
50, 1661
20, 1661
10, 1629
5, 1567
2, 1589
1, 1601

In [17]:
start = time.time()
goto = goto_arr_main[0][1].copy();goback = goto_arr_main[0][2].copy()
while True:
    diff = 0;v1 = -1;v2 = -1
    for i in range(n):
        for j in range(i+1,n):
            if goto[i] != j and goto[j] != i:
                d1 = dis[i][goto[i]]+dis[j][goto[j]]
                d2 = dis[i][j]+dis[goto[i]][goto[j]]
                if diff < d1-d2:
                    v1 = i;v2 = j;diff = d1-d2
    if diff == 0:
        break
    else:
        gt1 = goto[v1];gt2 = goto[v2]
        x = v1;y = v2
        while x != gt1:
            goto[x] = y;var = goback[y];goback[y] = x
            x = y;y = var
        goto[gt1] = gt2;goback[gt2] = gt1
print(time.time()-start,travel(goto),goto)

0.4520759582519531 1623.600192039111 [21, 54, 72, 81, 10, 97, 4, 89, 6, 50, 79, 47, 42, 78, 55, 69, 99, 30, 87, 13, 11, 80, 19, 28, 0, 49, 70, 17, 12, 48, 38, 63, 37, 86, 76, 52, 33, 20, 44, 84, 94, 67, 14, 65, 58, 51, 91, 9, 25, 59, 27, 2, 39, 71, 31, 77, 95, 88, 90, 32, 73, 34, 5, 57, 92, 1, 40, 62, 35, 64, 93, 68, 15, 82, 85, 41, 98, 66, 46, 61, 7, 16, 43, 24, 8, 29, 53, 96, 45, 56, 75, 36, 74, 18, 83, 26, 3, 22, 23, 60]


In [17]:
# getteing a random goto array
rand_arr = [i for i in range(n)]
rd.shuffle(rand_arr)
goto = [-1]*n
goback = [-1]*n
for i in range(n-1):
    goto[rand_arr[i]] = rand_arr[i+1]
    goback[rand_arr[i+1]] = rand_arr[i]
goto[rand_arr[n-1]] = rand_arr[0]
goback[rand_arr[0]] = rand_arr[n-1]
check_goto(goto)

True

In [25]:
# Savings algorithm
ver = [i for i in range(n)]
rd.shuffle(ver)
s = ver[0]
cost = []
for i in range(n):
    for j in range(i+1,n):
        if i != s and j != s:
            cost.append((dis[i][s]+dis[j][s]-dis[i][j],i,j))
cost.sort(reverse=True)
chain = [[] for i in range(n)]
parent = [i for i in range(n)]
rank = [0]*n
e = 0;i = 0
while e < n-2:
    if len(chain[cost[i][1]]) < 2 and len(chain[cost[i][2]]) < 2:
        if union(cost[i][1],cost[i][2],parent,rank):
            chain[cost[i][1]].append(cost[i][2])
            chain[cost[i][2]].append(cost[i][1])
            e+=1
    i+=1
for i in range(n):
    if i!=s and len(chain[i]) < 2:
        chain[i].append(s)
        chain[s].append(i)
goto = [-1]*n
goback = [-1]*n
par = s
ch = chain[s][0]
while ch != s:
    goto[par] = ch
    goback[ch] = par
    x = chain[ch][0]
    if x == par:
        x = chain[ch][1]
    par = ch
    ch = x
goto[par] = ch
goback[ch] = par
print(travel(goto),check_goto(goto))

1680.4053193173006 True


In [26]:
# greedy edge selection
cost = []
for i in range(n):
    for j in range(i+1,n):
        cost.append((dis[i][j],i,j))
cost.sort()
parent = [i for i in range(n)]
rank = [0]*n
chain = [[] for i in range(n)]
e = 0;i = 0
while e < n-1:
    if len(chain[cost[i][1]]) < 2 and len(chain[cost[i][2]]) < 2:
        if union(cost[i][1],cost[i][2],parent,rank):
            chain[cost[i][1]].append(cost[i][2])
            chain[cost[i][2]].append(cost[i][1])
            e+=1
    i+=1
x = -1
for i in range(n):
    if len(chain[i]) < 2:
        if x == -1:
            x = i
        else:
            chain[x].append(i)
            chain[i].append(x)
goto = [-1]*n
goback = [-1]*n
par = s
ch = chain[s][0]
while ch != s:
    goto[par] = ch
    goback[ch] = par
    x = chain[ch][0]
    if x == par:
        x = chain[ch][1]
    par = ch
    ch = x
goto[par] = ch
goback[ch] = par
print(travel(goto),check_goto(goto))

1644.5538288971502 True
